In [93]:
sample_master = {
    "item": {
        "name": "Demo",
        "price": 10,
        "is_active": True,
        "tag": "Soap",
        "godown_id": 1
    }
}

In [168]:
sample_sales = {
    "sales": {
        "customer_id": 1,
        "__attach__": [
                {
                    "sales_item": {
                        "item_id": 1,
                        "qty": 10,
                        "price": 20,
                        "total": 30
                    }
                },
                {
                        "sales_item": {
                            "item_id": 2,
                            "qty": 100,
                            "price": 200,
                            "total": 300
                        }
                    }
            ]
        }
    }

In [169]:
godown = {
    "cols": {
        "name": str,
        "location": str
    },
    "required": ['name']
}

In [170]:
item = {
     "cols": {
        "name": str,
        "price": int,
         "is_active": bool,
         "tag": str,
         "godown_id": int
     },
     "required": ['name', 'price', 'tag'],
 }

In [177]:
sales = {
    "cols": {
       "customer_id": int,
        "__attach__": ["sales_item"]
    },
    "required": ["customer_id"]
}

In [172]:
sales_item = {
    "cols": {
        "sales_id": int,
        "item_id": int,
        "qty": float,
        "price": float,
        "total": float
    },
    "required": []
}

In [173]:
repo = {
    "item": item,
    "sales": sales,
    "sales_item": sales_item
}

In [174]:
sample_master

{'item': {'name': 'Demo',
  'price': 10,
  'is_active': True,
  'tag': 'Soap',
  'godown_id': 1}}

In [180]:
class Repository:
    def __init__(self, **kwargs):
        self.repo = list(kwargs.keys())[0]
        self.cols = []
        self.values = []
        self.items = kwargs[self.repo]
        self.dtypes = [str, int, float, bool]
        self.is_child = False
    
    def insert(self):
        self.__check_required_cols()
        if not self.is_child:
            print("Transaction begin")
        for k in self.items:
            if repo[self.repo]['cols'][k] in self.dtypes and k not in ['__attach__']:
                self.cols.append(k)
                if type(self.items[k]) in [str, bool]:
                    self.values.append(f'"{self.items[k]}"')
                else:
                    self.values.append(self.items[k])   
        if "__attach__" not in self.items:
            return self.create_query()
        else:
            id = self.create_query()
            for attach in self.items['__attach__']:
                for attach_item in attach:
                    attach[attach_item][f"{self.repo}_id"] = id
                    attach_repo = {
                        attach_item: attach[attach_item]
                    }
                    aa = Repository(**attach_repo)
                    aa.is_child = True
                    aa.insert()
        if not self.is_child:
            print("Transaction Ends")
        
    def __check_required_cols(self):
        for req_col in repo[self.repo]['required']:
            if req_col not in self.items:
                raise Exception(f"Column {req_col} missing")

                
    def create_query(self):
        query = f"""
                INSERT INTO {self.repo} 
                    ({",".join(self.cols)})
                VALUES 
                    ({",".join(map(str, self.values))})
                """
        print(query)
        return 1
    

In [181]:
ma = Repository(**sample_sales)
ma.insert()

Transaction begin

                INSERT INTO sales 
                    (customer_id)
                VALUES 
                    (1)
                

                INSERT INTO sales_item 
                    (item_id,qty,price,total,sales_id)
                VALUES 
                    (1,10,20,30,1)
                

                INSERT INTO sales_item 
                    (item_id,qty,price,total,sales_id)
                VALUES 
                    (2,100,200,300,1)
                
Transaction Ends


In [ ]:
ma = Repository(**sample_master)
ma.insert()